In [1]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored
import wandb

In [2]:
def set_all_seeds():
    """Set all seeds"""
    torch.manual_seed(0)
    np.random.seed(8)
    os.environ['PYTHONHASHSEED']=str(0)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False


In [3]:
import dataset_handler as dh
import helper_func as hf

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

In [4]:
"""Load gsm8k"""
#priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt_codegen.txt" # for codegen
priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt.txt" # for gpt-j
current_dataset = dh.init_dataset_from_name("gsm8k", primingtext_path = priming_text_path)

In [5]:
"""Load asdiv"""
#priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt" # for codegen
priming_text_path = "data/priming_texts/asdiv/asdiv_prefix.txt" # for gpt-j
current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

In [6]:
set_all_seeds()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(100)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Write a program that prints the answer to the following question. When they got there, they saw that SFL is already filled with people. Every entrance has a long line, with 283 people waiting to get in. If SFL has 5 entrances, how many people in total are waiting to get in?
1415 (people)


In [6]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
#model_args.model = "codegen-350M-mono"
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 467.01s
loading tokenizer
loading tokenizer took 4.00s


In [7]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [8]:
config = hf.codegen_gen_args()
config.num_return_sequences = 5 # 4 for gsm8k 5 for asdiv
config.k = 3
config.max_lenght_after_input = 250
config.top_p = 0.95
config.top_k = 50
config.temperature = 0.7
config.min_length = 3

config = hf.gptj_gen_args()
config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv

In [9]:

set_all_seeds()
hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, print_output=False)

TESTING STARTED
@sample 5 -> Pass@3 = 0.3
@sample 10 -> Pass@3 = 0.25


KeyboardInterrupt: 

In [9]:
with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name="@100-asdiv-gpt_j"):

        set_all_seeds()
        pass_at_k = hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

        wandb.log({"pass_at_k": pass_at_k})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antoniolopardo (use `wandb login --relogin` to force relogin)


TESTING STARTED
@sample 5 -> Pass@3 = 0.24
@sample 10 -> Pass@3 = 0.21000000000000002
@sample 15 -> Pass@3 = 0.15
@sample 20 -> Pass@3 = 0.1105263157894737
@sample 25 -> Pass@3 = 0.15
@sample 30 -> Pass@3 = 0.17586206896551723
@sample 35 -> Pass@3 = 0.1676470588235294
@sample 40 -> Pass@3 = 0.18717948717948715
@sample 45 -> Pass@3 = 0.18863636363636366
@sample 50 -> Pass@3 = 0.19387755102040813
@sample 55 -> Pass@3 = 0.187037037037037
@sample 60 -> Pass@3 = 0.17413793103448277
@sample 65 -> Pass@3 = 0.16984126984126985
@sample 70 -> Pass@3 = 0.17941176470588235
@sample 75 -> Pass@3 = 0.16712328767123286
@sample 80 -> Pass@3 = 0.1564102564102564
@sample 85 -> Pass@3 = 0.1542168674698795
@sample 90 -> Pass@3 = 0.16363636363636364
@sample 95 -> Pass@3 = 0.17282608695652174
@sample 100 -> Pass@3 = 0.17422680412371133


Pass@3 = 0.17422680412371133



pass_at_k,▁
pass_at_k,0.17423
